In [1]:
import numpy as np  # Linear algebra.
import pandas as pd # Data processing, CSV file I/O (e.g. pd.read_csv).
import os           # File management.

# Input data files are available in the "../input/" directory.

print("Setup complete.")

Setup complete.


# Time Series Notebook.

This notebook aims to compute and display time series of the number of cells alive during the experiments.

## Importing dataset file names in a DataFrame to compute time series.

In [2]:
# This cell is not usefull on kaggle.com

print(os.getcwd())  
# If you're not using this notebook on kaggle.com :
#   Result should end with "/ecn_radiobiology/notebook".
#   If it is not the case, you're running the notebook on the wrong folder.

/home/momo/Documents/Centrale/EI3/PAPPL/ecn_radiobiology/notebook


In [3]:
# Path of the dataset :

path = '../input/glioblastoma-radiosensitivity-dataset/glioblastoma_data/extracted_data/cell_behaviors/videos_transcriptions_csv'
print(path)

../input/glioblastoma-radiosensitivity-dataset/glioblastoma_data/extracted_data/cell_behaviors/videos_transcriptions_csv


In [4]:
def import_files(path):
    struc = pd.DataFrame(columns =['Radiation level', 'Path','codeFile'])
    read = os.walk(path)
    for root, dirs, files in read:
        for name in files:
            if name.endswith(".csv"):
                #print(name)
                radiation = name.split('gy')
                filepath = path+ '/' + name
                codeFile = name[:-4] #removing suffix
                
                struc = struc.append({'Radiation level' : radiation[0], 'Path' : filepath, 'codeFile' : codeFile}, ignore_index=True)
                
    return struc

In [5]:
# Importing the path of each files of the dataset in a DataFrame.

dataset = import_files(path)

dataset.head()

,Radiation level,Path,codeFile
0,10,../input/glioblastoma-radiosensitivity-dataset...,10gy_n3_07-06-2021-10Gy-u251-5B
1,10,../input/glioblastoma-radiosensitivity-dataset...,10gy_n2_2021n210Gy6C
2,2,../input/glioblastoma-radiosensitivity-dataset...,2gy_n3_2021-04-2Gy-2C
3,4,../input/glioblastoma-radiosensitivity-dataset...,4gy_n1_160930.nd2u2514gyp1
4,15,../input/glioblastoma-radiosensitivity-dataset...,15gy_n2_2021N215GY5B
